In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import pickle
import warnings
warnings.filterwarnings('ignore')
pd.options.display.max_columns = None

from sklearn.model_selection import KFold, StratifiedKFold, train_test_split
from sklearn.metrics import roc_auc_score, accuracy_score, classification_report, recall_score, precision_score, f1_score
from sklearn.ensemble import RandomForestClassifier
import df_maker


In [2]:
!ls ../data/pickles

df_con_class_nodates.pickle  df_sparse_con_subtype.pickle
df_full.pickle


# iterate through dfs

In [3]:
df = pickle.load(open('../data/pickles/df_full.pickle', 'rb'))
df_dict = {'no_targ_pop':df_maker.no_targ_pop(df),'no_years':df_maker.no_years(df), 'no_years_no_target': df_maker.no_years_no_target(df),
           'subtype':df_maker.conv_subt(df), 'subtype_no_targ':df_maker.subt_no_target(df), 
           'subt_no_tar_no_year':df_maker.subt_no_targ_no_years(df),
           'class':df_maker.conv_class(df), 'class_no_target':df_maker.class_no_target(df),
           'type':df_maker.conv_type(df), 'type_no_target':df_maker.type_no_target(df)}
best_mean_roc_auc = {}
roc_auc_list = []

In [5]:
for key, value in df_dict.items():
    
    df = value
    X = df[0]
    y = df[1]

    X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = .2, random_state=42)
    from imblearn.over_sampling import RandomOverSampler
    roc_auc_list = []
    kf = StratifiedKFold(n_splits=5, random_state=42)

    for train_ind, val_ind in kf.split(X_train, y_train):

        X_ktrain, X_kval = X.iloc[train_ind], X.iloc[val_ind]
        y_ktrain, y_kval = y.iloc[train_ind], y.iloc[val_ind]

        ros = RandomOverSampler(random_state=42)
        X_train_resampled, y_train_resampled = ros.fit_resample(X_ktrain, y_ktrain)

        reg = RandomForestClassifier()
        reg.fit(X_train_resampled,y_train_resampled)
        y_pred = reg.predict_proba(X_kval)[:,1]
        score = roc_auc_score(y_kval, y_pred)

        roc_auc_list.append(score)

    best_mean_roc_auc[key] = sum(roc_auc_list)/len(roc_auc_list)

    #print(sum(roc_auc_list)/len(roc_auc_list))
print(best_mean_roc_auc)


{'no_targ_pop': 0.5287560753018639, 'no_years': 0.5635009772106405, 'no_years_no_target': 0.5700442655348998, 'subtype': 0.5488733951059451, 'subtype_no_targ': 0.5440490932994081, 'subt_no_tar_no_year': 0.5844250889096927, 'class': 0.590644563397176, 'class_no_target': 0.5696955422895469, 'type': 0.6009174593468518, 'type_no_target': 0.5834896043851293}


In [6]:
df = pickle.load(open('../data/pickles/df_full.pickle', 'rb'))
df = df_maker.conv_type(df)
X = df[0]
y = df[1]

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = .2, random_state=42)
roc_auc_list = []

best_depth = 0
best_depth_roc = 0
for depth in range(2,20):
    kf = StratifiedKFold(n_splits=5, random_state=42)
    roc_auc_list = []
    for train_ind, val_ind in kf.split(X_train, y_train):

        X_train, X_val = X.iloc[train_ind], X.iloc[val_ind]
        y_train, y_val = y.iloc[train_ind], y.iloc[val_ind]

        ros = RandomOverSampler(random_state=42)
        X_train_resampled, y_train_resampled = ros.fit_resample(X_train, y_train)

        rf = RandomForestClassifier(max_depth=depth)
        rf.fit(X_train_resampled,y_train_resampled)
        y_pred = rf.predict_proba(X_val)[:,1]

        roc_auc_list.append(roc_auc_score(y_val, y_pred))
        
    mean_roc = sum(roc_auc_list)/len(roc_auc_list)
    if mean_roc > best_depth_roc:
        best_depth = depth
        best_depth_roc = mean_roc

print(best_depth, best_depth_roc)

3 0.6179685711643123


In [7]:
df = pickle.load(open('../data/pickles/df_full.pickle', 'rb'))
df = df_maker.conv_type(df)
X = df[0]
y = df[1]

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = .2, random_state=42)
roc_auc_list = []

best = 0
best_roc = 0
for feature_test in range(2,15):
    kf = StratifiedKFold(n_splits=5, random_state=42)
    roc_auc_list = []
    for train_ind, val_ind in kf.split(X_train, y_train):

        X_train, X_val = X.iloc[train_ind], X.iloc[val_ind]
        y_train, y_val = y.iloc[train_ind], y.iloc[val_ind]

        ros = RandomOverSampler(random_state=42)
        X_train_resampled, y_train_resampled = ros.fit_resample(X_train, y_train)

        rf = RandomForestClassifier(max_depth=5, max_features=15)
        rf.fit(X_train_resampled,y_train_resampled)
        y_pred = rf.predict_proba(X_val)[:,1]

        roc_auc_list.append(roc_auc_score(y_val, y_pred))
        
    mean_roc = sum(roc_auc_list)/len(roc_auc_list)
    if mean_roc > best_roc:
        best = feature_test
        best_roc = mean_roc

print(best, best_roc)

5 0.6174833789151164


In [10]:
## Max Depth 5, Max Features 5

In [11]:
df = pickle.load(open('../data/pickles/df_full.pickle', 'rb'))
df = df_maker.conv_type(df)
X = df[0]
y = df[1]

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = .2, random_state=42)
roc_auc_list = []


kf = StratifiedKFold(n_splits=5, random_state=42)
roc_auc_list = []
for train_ind, val_ind in kf.split(X_train, y_train):

    X_train, X_val = X.iloc[train_ind], X.iloc[val_ind]
    y_train, y_val = y.iloc[train_ind], y.iloc[val_ind]

    ros = RandomOverSampler(random_state=42)
    X_train_resampled, y_train_resampled = ros.fit_resample(X_train, y_train)

    rf = RandomForestClassifier(max_depth=5, max_features=5)
    rf.fit(X_train_resampled,y_train_resampled)
    y_pred = rf.predict_proba(X_val)[:,1]

    roc_auc_list.append(roc_auc_score(y_val, y_pred))

mean_roc = sum(roc_auc_list)/len(roc_auc_list)
print(mean_roc)

0.6057174341234758


# Manual Feature Explore

## All Features

In [4]:
df = pickle.load(open('../data/pickles/df_full.pickle', 'rb'))

In [5]:
best_mean_roc_auc = {}

In [6]:
X = df.drop(columns = ['recid_flag'])
y = df['recid_flag']

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = .2, random_state=42)
from imblearn.over_sampling import RandomOverSampler
roc_auc_list = []
kf = StratifiedKFold(n_splits=5, random_state=42)

for train_ind, val_ind in kf.split(X_train, y_train):

    X_ktrain, X_kval = X.iloc[train_ind], X.iloc[val_ind]
    y_ktrain, y_kval = y.iloc[train_ind], y.iloc[val_ind]
    
    ros = RandomOverSampler(random_state=42)
    X_train_resampled, y_train_resampled = ros.fit_resample(X_ktrain, y_ktrain)
    
    rf = RandomForestClassifier()
    rf.fit(X_train_resampled,y_train_resampled)
    y_pred = rf.predict(X_kval)
    score = roc_auc_score(y_kval, y_pred)
    
    roc_auc_list.append(score)

best_mean_roc_auc['full_feature'] = sum(roc_auc_list)/len(roc_auc_list)
print(sum(roc_auc_list)/len(roc_auc_list))

0.5317897266879001


In [8]:
import operator

df_feature_imp = {}

for imp, feature in (zip(rf.feature_importances_, list(X))):
    df_feature_imp[feature] = imp

sorted_features = sorted(df_feature_imp.items(), key=operator.itemgetter(1)) 
sorted_features

[('year_released_2015', 0.0),
 ('reporting_year_2018', 0.0),
 ('conviction_class_Other Felony (Old Code)', 2.336513780725313e-05),
 ('conviction_class_A Felony', 2.7916757359059834e-05),
 ('conviction_class_Simple Misdemeanor', 3.1981626383400434e-05),
 ('conviction_class_Sexual Predator Community Supervision',
  4.4894684806085164e-05),
 ('conviction_subtype_Animals', 0.00013419221975966483),
 ('conviction_class_Other Felony', 0.00013618400204200943),
 ('conviction_class_Felony - Mandatory Minimum', 0.0001563308570251822),
 ('reporting_year_2017', 0.0003074782229089708),
 ('year_released_2014', 0.0003714936947475633),
 ('conviction_subtype_Special Sentence Revocation', 0.000810921973361567),
 ('release_type_Paroled to Detainer - U.S. Marshall', 0.00099857096917253),
 ('conviction_subtype_Prostitution/Pimping', 0.0010815046789366422),
 ('conviction_subtype_Stolen Property', 0.0011921523749036595),
 ('conviction_class_Special Sentence 2005', 0.0012895611649751612),
 ('release_type_Parol

## No Dates

In [9]:
df = df_maker.no_years(df)

In [10]:
X = df[0]
y = df[1]

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = .2, random_state=42)
from imblearn.over_sampling import RandomOverSampler
roc_auc_list = []
kf = StratifiedKFold(n_splits=5, random_state=42)

for train_ind, val_ind in kf.split(X_train, y_train):

    X_ktrain, X_kval = X.iloc[train_ind], X.iloc[val_ind]
    y_ktrain, y_kval = y.iloc[train_ind], y.iloc[val_ind]
    
    ros = RandomOverSampler(random_state=42)
    X_train_resampled, y_train_resampled = ros.fit_resample(X_ktrain, y_ktrain)
    
    rf = RandomForestClassifier()
    rf.fit(X_train_resampled,y_train_resampled)
    y_pred = rf.predict(X_kval)
    score = roc_auc_score(y_kval, y_pred)
    
    roc_auc_list.append(score)

best_mean_roc_auc['full_feature_no_years'] = sum(roc_auc_list)/len(roc_auc_list)
print(sum(roc_auc_list)/len(roc_auc_list))

0.5521282047650967


In [12]:
import operator

df_feature_imp = {}

for imp, feature in (zip(rf.feature_importances_, list(X))):
    df_feature_imp[feature] = imp

sorted_features = sorted(df_feature_imp.items(), key=operator.itemgetter(1)) 
sorted_features

[('conviction_class_Other Felony (Old Code)', 1.3210271931708404e-06),
 ('conviction_class_A Felony', 1.5703713149512414e-05),
 ('conviction_class_Simple Misdemeanor', 8.78187557576308e-05),
 ('conviction_class_Sexual Predator Community Supervision',
  0.00011622660524844633),
 ('conviction_subtype_Animals', 0.0002630613708386635),
 ('conviction_class_Other Felony', 0.0003015137201505825),
 ('conviction_class_Felony - Mandatory Minimum', 0.0003084555712183155),
 ('conviction_class_Special Sentence 2005', 0.001329270607826163),
 ('conviction_subtype_Stolen Property', 0.001432721283737547),
 ('release_type_Paroled to Detainer - Iowa', 0.0015580367146415955),
 ('release_type_Paroled to Detainer - INS', 0.0015727966263432952),
 ('release_type_Paroled to Detainer - U.S. Marshall', 0.0015894775382019224),
 ('conviction_subtype_Prostitution/Pimping', 0.0016799105036429964),
 ('release_type_Released to Special Sentence', 0.0018122669980846506),
 ('conviction_subtype_Special Sentence Revocation

## No years no target pop flag

In [34]:
df = pickle.load(open('../data/pickles/df_full.pickle', 'rb'))
df = df_maker.no_years_no_target(df)
X = df[0]
y = df[1]

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = .2, random_state=42)
from imblearn.over_sampling import RandomOverSampler
roc_auc_list = []
kf = StratifiedKFold(n_splits=5, random_state=42)

for train_ind, val_ind in kf.split(X_train, y_train):

    X_ktrain, X_kval = X.iloc[train_ind], X.iloc[val_ind]
    y_ktrain, y_kval = y.iloc[train_ind], y.iloc[val_ind]
    
    ros = RandomOverSampler(random_state=42)
    X_train_resampled, y_train_resampled = ros.fit_resample(X_ktrain, y_ktrain)
    
    rf = RandomForestClassifier()
    rf.fit(X_train_resampled,y_train_resampled)
    y_pred = rf.predict_proba(X_kval)[:,1]
    score = roc_auc_score(y_kval, y_pred)
    
    roc_auc_list.append(score)

best_mean_roc_auc['no_years_no_target'] = sum(roc_auc_list)/len(roc_auc_list)

print(sum(roc_auc_list)/len(roc_auc_list))

0.5675595011400788


## No Target Flag

In [14]:
df = pickle.load(open('../data/pickles/df_full.pickle', 'rb'))
df = df_maker.no_targ_pop(df)
X = df[0]
y = df[1]

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = .2, random_state=42)
from imblearn.over_sampling import RandomOverSampler
roc_auc_list = []
kf = StratifiedKFold(n_splits=5, random_state=42)

for train_ind, val_ind in kf.split(X_train, y_train):

    X_ktrain, X_kval = X.iloc[train_ind], X.iloc[val_ind]
    y_ktrain, y_kval = y.iloc[train_ind], y.iloc[val_ind]
    
    ros = RandomOverSampler(random_state=42)
    X_train_resampled, y_train_resampled = ros.fit_resample(X_ktrain, y_ktrain)
    
    rf = RandomForestClassifier()
    rf.fit(X_train_resampled,y_train_resampled)
    y_pred = rf.predict(X_kval)
    score = roc_auc_score(y_kval, y_pred)
    
    roc_auc_list.append(score)
    
best_mean_roc_auc['full_feature_no_target'] = sum(roc_auc_list)/len(roc_auc_list)
print(sum(roc_auc_list)/len(roc_auc_list))

0.5235114643020122


## Conviction Class

In [16]:
df = pickle.load(open('../data/pickles/df_full.pickle', 'rb'))
df = df_maker.conv_class(df)
X = df[0]
y = df[1]

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = .2, random_state=42)
from imblearn.over_sampling import RandomOverSampler
roc_auc_list = []
kf = StratifiedKFold(n_splits=5, random_state=42)

for train_ind, val_ind in kf.split(X_train, y_train):

    X_ktrain, X_kval = X.iloc[train_ind], X.iloc[val_ind]
    y_ktrain, y_kval = y.iloc[train_ind], y.iloc[val_ind]
    
    ros = RandomOverSampler(random_state=42)
    X_train_resampled, y_train_resampled = ros.fit_resample(X_ktrain, y_ktrain)
    
    rf = RandomForestClassifier()
    rf.fit(X_train_resampled,y_train_resampled)
    y_pred = rf.predict(X_kval)
    score = roc_auc_score(y_kval, y_pred)
    
    roc_auc_list.append(score)

best_mean_roc_auc['conv_class'] = sum(roc_auc_list)/len(roc_auc_list)

print(sum(roc_auc_list)/len(roc_auc_list))

0.5672464741220502


## Conviction Class No Target Pop

In [17]:
df = pickle.load(open('../data/pickles/df_full.pickle', 'rb'))
df = df_maker.class_no_target(df)
X = df[0]
y = df[1]

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = .2, random_state=42)
from imblearn.over_sampling import RandomOverSampler
roc_auc_list = []
kf = StratifiedKFold(n_splits=5, random_state=42)

for train_ind, val_ind in kf.split(X_train, y_train):

    X_ktrain, X_kval = X.iloc[train_ind], X.iloc[val_ind]
    y_ktrain, y_kval = y.iloc[train_ind], y.iloc[val_ind]
    
    ros = RandomOverSampler(random_state=42)
    X_train_resampled, y_train_resampled = ros.fit_resample(X_ktrain, y_ktrain)
    
    rf = RandomForestClassifier()
    rf.fit(X_train_resampled,y_train_resampled)
    y_pred = rf.predict(X_kval)
    score = roc_auc_score(y_kval, y_pred)
    
    roc_auc_list.append(score)
    
best_mean_roc_auc['con_class_no_target'] = sum(roc_auc_list)/len(roc_auc_list)
print(sum(roc_auc_list)/len(roc_auc_list))

0.5434516247364696


## Conviction Type

In [18]:
df = pickle.load(open('../data/pickles/df_full.pickle', 'rb'))
df = df_maker.conv_type(df)
X = df[0]
y = df[1]

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = .2, random_state=42)
from imblearn.over_sampling import RandomOverSampler
roc_auc_list = []
kf = StratifiedKFold(n_splits=5, random_state=42)

for train_ind, val_ind in kf.split(X_train, y_train):

    X_ktrain, X_kval = X.iloc[train_ind], X.iloc[val_ind]
    y_ktrain, y_kval = y.iloc[train_ind], y.iloc[val_ind]
    
    ros = RandomOverSampler(random_state=42)
    X_train_resampled, y_train_resampled = ros.fit_resample(X_ktrain, y_ktrain)
    
    rf = RandomForestClassifier()
    rf.fit(X_train_resampled,y_train_resampled)
    y_pred = rf.predict(X_kval)
    score = roc_auc_score(y_kval, y_pred)
    
    roc_auc_list.append(score)
    
best_mean_roc_auc['con_type'] = sum(roc_auc_list)/len(roc_auc_list)
print(sum(roc_auc_list)/len(roc_auc_list))

0.5720941803718126


## Conviction Type No Target

In [19]:
df = pickle.load(open('../data/pickles/df_full.pickle', 'rb'))
df = df_maker.type_no_target(df)
X = df[0]
y = df[1]

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = .2, random_state=42)
from imblearn.over_sampling import RandomOverSampler
roc_auc_list = []
kf = StratifiedKFold(n_splits=5, random_state=42)

for train_ind, val_ind in kf.split(X_train, y_train):

    X_ktrain, X_kval = X.iloc[train_ind], X.iloc[val_ind]
    y_ktrain, y_kval = y.iloc[train_ind], y.iloc[val_ind]
    
    ros = RandomOverSampler(random_state=42)
    X_train_resampled, y_train_resampled = ros.fit_resample(X_ktrain, y_ktrain)
    
    rf = RandomForestClassifier()
    rf.fit(X_train_resampled,y_train_resampled)
    y_pred = rf.predict(X_kval)
    score = roc_auc_score(y_kval, y_pred)
    
    roc_auc_list.append(score)

best_mean_roc_auc['con_type_no_target'] = sum(roc_auc_list)/len(roc_auc_list)

print(sum(roc_auc_list)/len(roc_auc_list))

0.5657696473657061


## Conviction Subtype

In [20]:
df = pickle.load(open('../data/pickles/df_full.pickle', 'rb'))
df = df_maker.conv_subt(df)
X = df[0]
y = df[1]

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = .2, random_state=42)
from imblearn.over_sampling import RandomOverSampler
roc_auc_list = []
kf = StratifiedKFold(n_splits=5, random_state=42)

for train_ind, val_ind in kf.split(X_train, y_train):

    X_ktrain, X_kval = X.iloc[train_ind], X.iloc[val_ind]
    y_ktrain, y_kval = y.iloc[train_ind], y.iloc[val_ind]
    
    ros = RandomOverSampler(random_state=42)
    X_train_resampled, y_train_resampled = ros.fit_resample(X_ktrain, y_ktrain)
    
    rf = RandomForestClassifier()
    rf.fit(X_train_resampled,y_train_resampled)
    y_pred = rf.predict(X_kval)
    score = roc_auc_score(y_kval, y_pred)
    
    roc_auc_list.append(score)

best_mean_roc_auc['con_subtype'] = sum(roc_auc_list)/len(roc_auc_list)

print(sum(roc_auc_list)/len(roc_auc_list))

0.5381432928250558


### Subtype no target

In [21]:
df = pickle.load(open('../data/pickles/df_full.pickle', 'rb'))
df = df_maker.subt_no_target(df)
X = df[0]
y = df[1]

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = .2, random_state=42)
from imblearn.over_sampling import RandomOverSampler
roc_auc_list = []
kf = StratifiedKFold(n_splits=5, random_state=42)

for train_ind, val_ind in kf.split(X_train, y_train):

    X_ktrain, X_kval = X.iloc[train_ind], X.iloc[val_ind]
    y_ktrain, y_kval = y.iloc[train_ind], y.iloc[val_ind]
    
    ros = RandomOverSampler(random_state=42)
    X_train_resampled, y_train_resampled = ros.fit_resample(X_ktrain, y_ktrain)
    
    rf = RandomForestClassifier()
    rf.fit(X_train_resampled,y_train_resampled)
    y_pred = rf.predict_proba(X_kval)[:,1]
    score = roc_auc_score(y_kval, y_pred)
    
    roc_auc_list.append(score)

best_mean_roc_auc['con_subtype_no_target'] = sum(roc_auc_list)/len(roc_auc_list)

print(sum(roc_auc_list)/len(roc_auc_list))

0.5436915491682518


In [22]:
import operator

df_feature_imp = {}

for imp, feature in (zip(rf.feature_importances_, list(X))):
    df_feature_imp[feature] = imp

sorted_features = sorted(df_feature_imp.items(), key=operator.itemgetter(1)) 
sorted_features

[('year_released_2015', 0.0),
 ('reporting_year_2018', 0.0),
 ('reporting_year_2017', 0.001270367926129876),
 ('conviction_subtype_Animals', 0.0019305160559775158),
 ('year_released_2014', 0.001958179940856661),
 ('conviction_subtype_Stolen Property', 0.0023706248059725764),
 ('conviction_subtype_Sex Offender Registry/Residency', 0.005347742657265268),
 ('conviction_subtype_Prostitution/Pimping', 0.0055606932915679905),
 ('conviction_subtype_Special Sentence Revocation', 0.008680675596665986),
 ('conviction_subtype_Kidnap', 0.008995023858870858),
 ('year_released_2013', 0.013051234879954679),
 ('conviction_subtype_Flight/Escape', 0.013305070230256186),
 ('reporting_year_2014', 0.013430417476172948),
 ('reporting_year_2013', 0.013455676233496522),
 ('conviction_subtype_Robbery', 0.014569418824757024),
 ('year_released_2010', 0.01461553335465647),
 ('year_released_2011', 0.014780984773111105),
 ('reporting_year_2016', 0.015027337644162295),
 ('conviction_subtype_Other Public Order', 0.01

## Subtype no target no years

In [32]:
df = pickle.load(open('../data/pickles/df_full.pickle', 'rb'))
df = df_maker.subt_no_target(df)
X = df[0]
y = df[1]

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = .2, random_state=42)
from imblearn.over_sampling import RandomOverSampler
roc_auc_list = []
kf = StratifiedKFold(n_splits=5, random_state=42)

for train_ind, val_ind in kf.split(X_train, y_train):

    X_ktrain, X_kval = X.iloc[train_ind], X.iloc[val_ind]
    y_ktrain, y_kval = y.iloc[train_ind], y.iloc[val_ind]
    
    ros = RandomOverSampler(random_state=42)
    X_train_resampled, y_train_resampled = ros.fit_resample(X_ktrain, y_ktrain)
    
    rf = RandomForestClassifier()
    rf.fit(X_train_resampled,y_train_resampled)
    y_pred = rf.predict_proba(X_kval)[:,1]
    score = roc_auc_score(y_kval, y_pred)
    
    roc_auc_list.append(score)

best_mean_roc_auc['subt_no_targ_no_years'] = sum(roc_auc_list)/len(roc_auc_list)

print(sum(roc_auc_list)/len(roc_auc_list))

0.535833175704734


In [24]:
import operator

df_feature_imp = {}

for imp, feature in (zip(rf.feature_importances_, list(X))):
    df_feature_imp[feature] = imp

sorted_features = sorted(df_feature_imp.items(), key=operator.itemgetter(1)) 
sorted_features

[('year_released_2015', 0.0),
 ('reporting_year_2018', 0.0),
 ('year_released_2014', 0.0008371153766856744),
 ('reporting_year_2017', 0.0014432825356373018),
 ('conviction_subtype_Animals', 0.0018301843934731762),
 ('conviction_subtype_Stolen Property', 0.0021476250766961284),
 ('conviction_subtype_Prostitution/Pimping', 0.006183501785678435),
 ('conviction_subtype_Sex Offender Registry/Residency', 0.0071909524580127514),
 ('conviction_subtype_Special Sentence Revocation', 0.007247564950289105),
 ('conviction_subtype_Kidnap', 0.009477805675708963),
 ('year_released_2012', 0.01095574896596921),
 ('year_released_2013', 0.01129336602691543),
 ('year_released_2010', 0.011878886721861664),
 ('year_released_2011', 0.012100931091844161),
 ('reporting_year_2014', 0.012198956452125851),
 ('conviction_subtype_Flight/Escape', 0.012934283949309436),
 ('reporting_year_2013', 0.01294498228518141),
 ('reporting_year_2015', 0.013194369813335369),
 ('reporting_year_2016', 0.01368334841619365),
 ('convi

In [25]:
best_mean_roc_auc

{'full_feature': 0.5317897266879001,
 'full_feature_no_years': 0.5521282047650967,
 'no_years_no_target': 0.5568718996885214,
 'full_feature_no_target': 0.5235114643020122,
 'conv_class': 0.5672464741220502,
 'con_class_no_target': 0.5434516247364696,
 'con_type': 0.5720941803718126,
 'con_type_no_target': 0.5657696473657061,
 'con_subtype': 0.5381432928250558,
 'con_subtype_no_target': 0.5436915491682518,
 'subt_no_targ_no_years': 0.5302292751934168}

# With subtype no target no years, iterate through RF depth

In [26]:
df = pickle.load(open('../data/pickles/df_full.pickle', 'rb'))
df = df_maker.conv_subt(df)
X = df[0]
y = df[1]

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = .2, random_state=42)
roc_auc_list = []

best_depth = 0
best_depth_roc = 0
for depth in range(2,20):
    kf = StratifiedKFold(n_splits=5, random_state=42)
    roc_auc_list = []
    for train_ind, val_ind in kf.split(X_train, y_train):

        X_train, X_val = X.iloc[train_ind], X.iloc[val_ind]
        y_train, y_val = y.iloc[train_ind], y.iloc[val_ind]

        ros = RandomOverSampler(random_state=42)
        X_train_resampled, y_train_resampled = ros.fit_resample(X_train, y_train)

        rf = RandomForestClassifier(max_depth=depth)
        rf.fit(X_train_resampled,y_train_resampled)
        y_pred = rf.predict(X_val)

        roc_auc_list.append(roc_auc_score(y_val, y_pred))
        
    mean_roc = sum(roc_auc_list)/len(roc_auc_list)
    if mean_roc > best_depth_roc:
        best_depth = depth
        best_depth_roc = mean_roc

print(best_depth, best_depth_roc)

4 0.5829068761390509


# Random Forest Trim unimportant features

In [27]:
df = pickle.load(open('../data/pickles/df_full.pickle', 'rb'))
df = df.drop(columns = ['release_type_Discharged - Expiration of Sentence',
 'release_type_Parole Granted',
 'release_type_Paroled to Detainer - INS',
 'release_type_Paroled to Detainer - Iowa',
 'release_type_Paroled to Detainer - Out of State',
 'release_type_Paroled to Detainer - U.S. Marshall',
 'release_type_Paroled w/Immediate Discharge',
 'release_type_Released to Special Sentence',
 'supervising_district_ISC'])

In [28]:
X = df.drop('recid_flag', axis = 1)
y= df['recid_flag']
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = .2, random_state=42)


In [29]:
roc_auc_list = []
for train_ind, val_ind in kf.split(X_train, y_train):

    X_train, X_val = X.iloc[train_ind], X.iloc[val_ind]
    y_train, y_val = y.iloc[train_ind], y.iloc[val_ind]
    
    ros = RandomOverSampler(random_state=42)
    X_train_resampled, y_train_resampled = ros.fit_resample(X_train, y_train)
    
    rf = RandomForestClassifier()
    rf.fit(X_train_resampled,y_train_resampled)
    y_pred = rf.predict(X_val)
    roc_auc_list.append(roc_auc_score(y_val, y_pred))

print(roc_auc_list)
print(sum(roc_auc_list)/len(roc_auc_list))

[0.5657270388218403, 0.5211698956709406, 0.5250119887648148, 0.5026952431763145, 0.5050404864613174]
0.5239289305790455


In [30]:
import operator

df_feature_imp = {}

for imp, feature in (zip(rf.feature_importances_, list(X))):
    df_feature_imp[feature] = imp

sorted_features = sorted(df_feature_imp.items(), key=operator.itemgetter(1)) 
sorted_features

[('year_released_2015', 0.0),
 ('reporting_year_2018', 0.0),
 ('conviction_class_A Felony', 0.0),
 ('conviction_class_Sexual Predator Community Supervision',
  4.663409089258192e-06),
 ('conviction_class_Other Felony (Old Code)', 1.1873848943455245e-05),
 ('conviction_class_Simple Misdemeanor', 7.81853300512634e-05),
 ('conviction_class_Felony - Mandatory Minimum', 0.00015381075909536162),
 ('conviction_class_Other Felony', 0.000154524102823699),
 ('conviction_subtype_Animals', 0.00018949486681110717),
 ('year_released_2014', 0.00030869335020244123),
 ('reporting_year_2017', 0.00047916668468760484),
 ('conviction_subtype_Special Sentence Revocation', 0.0007340929066488851),
 ('conviction_subtype_Stolen Property', 0.0012780158336033055),
 ('conviction_subtype_Prostitution/Pimping', 0.001405186791133628),
 ('conviction_class_Special Sentence 2005', 0.001626746442258532),
 ('conviction_subtype_Kidnap', 0.0016902746390038542),
 ('conviction_subtype_Flight/Escape', 0.0026742562099379154),
 

## Tune depth

In [31]:
best_depth = 0
best_depth_roc = 0
for depth in range(2,20):
    kf = StratifiedKFold(n_splits=5, random_state=42)
    roc_auc_list = []
    metric_list = [accuracy, precision,recall,  f1_scores]
    for train_ind, val_ind in kf.split(X_train, y_train):

        X_train, X_val = X.iloc[train_ind], X.iloc[val_ind]
        y_train, y_val = y.iloc[train_ind], y.iloc[val_ind]

        ros = RandomOverSampler(random_state=42)
        X_train_resampled, y_train_resampled = ros.fit_resample(X_train, y_train)

        rf = RandomForestClassifier(max_depth=depth)
        rf.fit(X_train_resampled,y_train_resampled)
        y_pred = rf.predict(X_val)

        roc_auc_list.append(roc_auc_score(y_val, y_pred))
        
    mean_roc = sum(roc_auc_list)/len(roc_auc_list)
    if mean_roc > best_depth_roc:
        best_depth = depth
        best_depth_roc = mean_roc

NameError: name 'accuracy' is not defined

In [ ]:
print(best_depth, best_depth_roc)

## Run with Best Depth of 

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = .2, random_state=42)

roc_auc_list = []
for train_ind, val_ind in kf.split(X_train, y_train):

    X_train, X_val = X.iloc[train_ind], X.iloc[val_ind]
    y_train, y_val = y.iloc[train_ind], y.iloc[val_ind]
    
    ros = RandomOverSampler(random_state=42)
    X_train_resampled, y_train_resampled = ros.fit_resample(X_train, y_train)
    print(len(X_train_resampled))
    rf = RandomForestClassifier(max_depth=best_depth)
    rf.fit(X_train_resampled,y_train_resampled)
    y_pred = rf.predict(X_val)
    roc_auc_list.append(roc_auc_score(y_val, y_pred))

print(roc_auc_list)
print(sum(roc_auc_list)/len(roc_auc_list))

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = .2, random_state=42)
ros = RandomOverSampler(random_state=42)
X_train_resampled, y_train_resampled = ros.fit_resample(X_train, y_train)

rf = RandomForestClassifier(max_depth = 4)
rf.fit(X_train_resampled, y_train_resampled)
y_pred = rf.predict(X_test)
print(roc_auc_score(y_test, y_pred))

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
rf_confusion = confusion_matrix(y_test, y_pred)

plt.figure(dpi=100)

target_names = ['no_recid', 'recid']
sns.heatmap(rf_confusion, cmap=plt.cm.Blues, annot=True, square=True,
           xticklabels=target_names,
           yticklabels=target_names,
           fmt='d')

plt.xlabel('Predicted recidivism')
plt.ylabel('Actual recidivism')

In [ ]:
import operator

df_feature_imp = {}

for imp, feature in (zip(rf.feature_importances_, list(X))):
    df_feature_imp[feature] = imp

sorted_features = sorted(df_feature_imp.items(), key=operator.itemgetter(1)) 
sorted_features

# DF Sparse pickle with subtypes

In [ ]:
df = pickle.load(open('../data/pickles/df_sparse_con_subtype.pickle', 'rb'))

In [ ]:
X = df.drop('recid_flag', axis = 1)
y = df.recid_flag

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = .2, random_state=42)

best_depth = 0
best_depth_roc = 0
for depth in range(2,20):
    kf = StratifiedKFold(n_splits=5, random_state=42)
    roc_auc_list = []
    metric_list = [accuracy, precision,recall,  f1_scores]
    for train_ind, val_ind in kf.split(X_train, y_train):

        X_train, X_val = X.iloc[train_ind], X.iloc[val_ind]
        y_train, y_val = y.iloc[train_ind], y.iloc[val_ind]

        ros = RandomOverSampler(random_state=42)
        X_train_resampled, y_train_resampled = ros.fit_resample(X_train, y_train)

        rf = RandomForestClassifier(max_depth=depth)
        rf.fit(X_train_resampled,y_train_resampled)
        y_pred = rf.predict(X_val)

        roc_auc_list.append(roc_auc_score(y_val, y_pred))
        
    mean_roc = sum(roc_auc_list)/len(roc_auc_list)
    if mean_roc > best_depth_roc:
        best_depth = depth
        best_depth_roc = mean_roc

In [ ]:
print(best_depth, best_depth_roc)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = .2, random_state=42)
ros = RandomOverSampler(random_state=42)
X_train_resampled, y_train_resampled = ros.fit_resample(X_train, y_train)

rf = RandomForestClassifier(max_depth = 3)
rf.fit(X_train_resampled, y_train_resampled)
y_pred = rf.predict(X_test)
print(roc_auc_score(y_test, y_pred))

In [ ]:
import operator

df_feature_imp = {}

for imp, feature in (zip(rf.feature_importances_, list(X))):
    df_feature_imp[feature] = imp

sorted_features = sorted(df_feature_imp.items(), key=operator.itemgetter(1)) 
sorted_features

## Cull with unimportant features

In [ ]:
X = df.drop(columns = ['recid_flag',
                       'race_American Indian or Alaska Native - Non-Hispanic', 
                         'conviction_subtype_Animals',
                         'conviction_subtype_Arson',
                         'conviction_subtype_Drug Possession', 
                         'conviction_subtype_Forgery/Fraud', 
                         'conviction_subtype_Other Drug', 
                         'conviction_subtype_Other Violent', 
                         'conviction_subtype_Prostitution/Pimping', 
                         'conviction_subtype_Sex Offender Registry/Residency', 
                         'conviction_subtype_Special Sentence Revocation', 
                         'conviction_subtype_Stolen Property', 
                         'conviction_subtype_Theft', 
                         'conviction_subtype_Traffic', 
                         'conviction_subtype_Vandalism', 
                         'conviction_subtype_Weapons',
                          'target_pop_flag'])
y = df.recid_flag

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = .2, random_state=42)

best_depth = 0
best_depth_roc = 0
for depth in range(2,20):
    kf = StratifiedKFold(n_splits=5, random_state=42)
    roc_auc_list = []
    metric_list = [accuracy, precision,recall,  f1_scores]
    for train_ind, val_ind in kf.split(X_train, y_train):

        X_train, X_val = X.iloc[train_ind], X.iloc[val_ind]
        y_train, y_val = y.iloc[train_ind], y.iloc[val_ind]

        ros = RandomOverSampler(random_state=42)
        X_train_resampled, y_train_resampled = ros.fit_resample(X_train, y_train)

        rf = RandomForestClassifier(max_depth=depth)
        rf.fit(X_train_resampled,y_train_resampled)
        y_pred = rf.predict(X_val)

        roc_auc_list.append(roc_auc_score(y_val, y_pred))
        
    mean_roc = sum(roc_auc_list)/len(roc_auc_list)
    if mean_roc > best_depth_roc:
        best_depth = depth
        best_depth_roc = mean_roc

In [ ]:
print(best_depth, best_depth_roc)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = .2, random_state=42)
ros = RandomOverSampler(random_state=42)
X_train_resampled, y_train_resampled = ros.fit_resample(X_train, y_train)

rf = RandomForestClassifier(max_depth = best_depth)
rf.fit(X_train_resampled, y_train_resampled)
y_pred = rf.predict(X_test)
print(roc_auc_score(y_test, y_pred))

In [ ]:
import operator

df_feature_imp = {}

for imp, feature in (zip(rf.feature_importances_, list(X))):
    df_feature_imp[feature] = imp

sorted_features = sorted(df_feature_imp.items(), key=operator.itemgetter(1)) 
for x in sorted_features:
    print(x)
    
print("\nROC_AUC_Score: %s" %roc_auc_score(y_test, y_pred))

In [ ]:
rf_confusion = confusion_matrix(y_test, y_pred)

plt.figure(dpi=100)

target_names = ['no_recid', 'recid']
sns.heatmap(rf_confusion, cmap=plt.cm.Blues, annot=True, square=True,
           xticklabels=target_names,
           yticklabels=target_names,
           fmt='d')

plt.xlabel('Predicted recidivism')
plt.ylabel('Actual recidivism')

# For Flask

In [ ]:
X = df.drop(columns = ['recid_flag'])
y = df['recid_flag']

In [ ]:
X = X[['race_Black - Non-Hispanic',
 'race_White - Hispanic',
 'race_White - Non-Hispanic',
 'age_at_release_25-34',
 'age_at_release_35-44',
 'age_at_release_45-54',
 'age_at_release_55 and Older',
 'age_at_release_Under 25',]]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = .2, random_state=42)
ros = RandomOverSampler(random_state=42)
X_train_resampled, y_train_resampled = ros.fit_resample(X_train, y_train)

rf = RandomForestClassifier(max_depth = 4)
rf.fit(X_train_resampled, y_train_resampled)
y_pred = rf.predict(X_test)
print(roc_auc_score(y_test, y_pred))

In [ ]:
import pickle
with open('flask_test.pickle', 'wb') as write_file:
    pickle.dump(rf, write_file)

In [ ]:
!ls

In [ ]:
list(X)